In [ ]:
!gdown  18cij6foP2ilw2NtadINSnYP6UUbenXCf

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 118MB/s]


In [ ]:
!unzip scc5966.zip

Archive:  scc5966.zip
  inflating: movie_reviews.csv       
  inflating: movies_data.csv         
  inflating: test_data.csv           
  inflating: train_data .csv         
  inflating: users_data.csv          


In [ ]:
import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')

In [ ]:
import pandas as pd
from tqdm import tqdm 

df=pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in tqdm(df_train_data['movie_id'].unique()):
  temp = df_train_data[df_train_data.movie_id == movie]
  df.loc[temp.user_id, movie] = temp.rating.to_list()

In [ ]:
from scipy.stats import pearsonr
import numpy as np

In [ ]:
cont = 1
dic_sim = {}

for user1 in tqdm(df.index):
  for user2 in df.index[cont:]:
    x = np.array(df.loc[user1], dtype=float)
    y = np.array(df.loc[user2], dtype=float)

    bad = ~np.logical_or(np.isnan(x), np.isnan(y))

    x = np.compress(bad, x)  
    y = np.compress(bad, y)  

    if len(x) > 1:
      similarity = pearsonr(x,y)[0]
      if similarity is np.nan:
        similarity = 0
    else:
      similarity = 0

    dic_sim[(user1,user2)] = similarity

  cont+=1

  0%|          | 0/3952 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
100%|██████████| 3952/3952 [1:45:41<00:00,  1.60s/it]


In [ ]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_users_pearson.p', 'wb') as fp:
    pickle.dump(dic_sim, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_users_pearson.p', 'rb') as fp:
    data = pickle.load(fp)

In [ ]:
def similarity_users(similarities, neighbor_user, user):
  
  if user > neighbor_user:
    similarity = similarities[(tuple([neighbor_user]),tuple([user]))]
  else:
    similarity = similarities[(tuple([user]),tuple([neighbor_user]))]
    
  return similarity

def k_neighbors_nearest(df, similarities, k, user, item):
  k_neighbors_similarity = [-2] * k
  k_neighbors = [-1] * k

  for neighbor_user in df.index:
    nu = neighbor_user
    if user != nu and df.loc[nu][item] is not np.nan:
      similarity = similarity_users(similarities, nu, user)

      for i in range(k):
        if similarity > k_neighbors_similarity[i]:
          aux = k_neighbors_similarity[i]
          k_neighbors_similarity[i] = similarity
          similarity = aux

          aux = k_neighbors[i]
          k_neighbors[i] = nu
          nu = aux

  return k_neighbors

def pred(df, similarities, k, user, item):

  user_mean = np.mean(df.loc[user])

  k_neighbors = k_neighbors_nearest(df, similarities, k, user, item)
  
  sum = 0
  sum_similarity = 0

  for neighbor_user in k_neighbors:
    if(neighbor_user != -1): #se nao deu o numero maximo de vizinhos mais proximos
      rating_neighbor_user = df.loc[neighbor_user][item]
      similarity = similarity_users(similarities, neighbor_user, user)
      mean_neighbor_user = np.mean(df.loc[neighbor_user])
      
      sum_similarity+= similarity
      
      sum+= similarity * (rating_neighbor_user - mean_neighbor_user)

  if sum_similarity == 0:
    return user_mean #se a soma das similaridades é 0, sobra somente a media do usuario
  else:
    return user_mean + (sum/sum_similarity)

In [ ]:
items_mean = np.mean(df)
items_mean

In [ ]:
user_mean = np.mean(df.T)
user_mean

In [ ]:
total_mean = np.nanmean(df.values.tolist()) 
total_mean

3.603814223642363

In [ ]:
k = 1
  
l= []
for item,row in tqdm(df_test_data.iterrows()):
  user = row['user_id']
  item = row['movie_id']
  if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
    l.append(items_mean[item])
  elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
    l.append(user_mean[user])
  elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
    l.append(total_mean)
  else:
    l.append(pred(df, data, k, user, item))
  
df_test_data['rating'] = l
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FC_byUsers_' + str(k) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)